In [1]:
bucket = 'ag5yy-sagemaker'
prefix = 'sagemaker/xgboost_credit_risk'

# Define IAM role
import boto3
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer

role = get_execution_role()

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls

--2021-04-21 14:10:04--  https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5539328 (5.3M) [application/x-httpd-php]
Saving to: ‘default of credit card clients.xls.2’

default of credit c 100%[===================>]   5.28M  7.21MB/s    in 0.7s    

2021-04-21 14:10:05 (7.21 MB/s) - ‘default of credit card clients.xls.2’ saved [5539328/5539328]



In [3]:
dataset = pd.read_excel('default of credit card clients.xls')
pd.set_option('display.max_rows', 8)
pd.set_option('display.max_columns', 15)
dataset

,Unnamed: 0,X1,X2,X3,X4,X5,X6,...,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,...,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,...,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,...,1518,1500,1000,1000,1000,5000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,29997,150000,1,3,2,43,-1,...,1837,3526,8998,129,0,0,0
29998,29998,30000,1,2,2,37,4,...,0,0,22000,4200,2000,3100,1
29999,29999,80000,1,3,1,41,1,...,85900,3409,1178,1926,52964,1804,1
30000,30000,50000,1,2,1,46,0,...,2078,1800,1430,1000,1000,1000,1


In [4]:
dataset = dataset.drop('Unnamed: 0', axis=1)
dataset = pd.concat([dataset['Y'], dataset.drop(['Y'], axis=1)], axis=1)

In [5]:
train_data, validation_data, test_data = np.split(dataset.sample(frac=1, random_state=1729), [int(0.7 * len(dataset)), int(0.9 * len(dataset))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

In [6]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

In [7]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}

sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(eta=0.1, objective='binary:logistic', num_round=25) 
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2021-04-21 14:10:09 Starting - Starting the training job...
2021-04-21 14:10:34 Starting - Launching requested ML instancesProfilerReport-1619014209: InProgress
......
2021-04-21 14:11:35 Starting - Preparing the instances for training.........
2021-04-21 14:12:58 Downloading - Downloading input data...
2021-04-21 14:13:36 Training - Downloading the training image...
2021-04-21 14:14:09 Uploading - Uploading generated training modelArguments: train
[2021-04-21:14:14:05:INFO] Running standalone xgboost training.
[2021-04-21:14:14:05:INFO] File size need to be processed in the node: 2.32mb. Available memory size in the node: 8390.56mb
[2021-04-21:14:14:05:INFO] Determined delimiter of CSV input is ','
[14:14:05] S3DistributionType set as FullyReplicated
[14:14:05] 21000x23 matrix with 483000 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-04-21:14:14:05:INFO] Determined delimiter of CSV input is ','
[14:14:05] S3DistributionType set as FullyReplic

In [8]:
xgb_predictor = xgb.deploy(
initial_instance_count = 1,
instance_type = 'ml.m4.xlarge',
serializer = CSVSerializer())

---------------!

In [9]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.to_numpy()[:,1:])
predictions

array([0.10145303, 0.29316297, 0.70782304, ..., 0.15233986, 0.38610485,
       0.10773233])

In [13]:
cm = pd.crosstab(index=test_data['Y'], columns=np.round(predictions), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Default", "Default"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Default", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Default", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 82.1%

Predicted      No Default      Default
Observed
No Default     84% (2245)    33% (107)
Default         16% (429)     67% (220) 

